<a href="https://colab.research.google.com/github/poet-developer/getPoemLineData_with-wikisource/blob/Multiple/%EC%9C%84%ED%82%A4%EB%AC%B8%ED%97%8C_%EB%8B%A4%EC%A4%91%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 위키문헌(Wikisource) XML파일 처리

In [1]:
pip install -q wikiextractor gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 1.6 MB/s eta 0:00:00


In [2]:
# 필요 패키지 load
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
# import gutenbergpy.textget
from pprint import pprint
import gdown
from collections import Counter
from itertools import chain

### 위키문헌 (wikisource)
https://dumps.wikimedia.org/kowikisource/20240901/  
참고문헌 : 서재현, 김병준, 김민우 and 박소정. (2021). 멀리서 읽는 “우리”― Word2Vec, N-gram을 이용한 근대 소설 텍스트 분석. 대동문화연구, 115, 349-386.

##### 위키문헌 전처리

In [3]:
# 위키문헌 xml 원본 파일 다운로드
# https://dumps.wikimedia.org/kowikisource/20240901/kowikisource-20240901-pages-articles.xml.bz2
!wget https://dumps.wikimedia.org/kowikisource/20241101/kowikisource-20241101-pages-articles.xml.bz2

--2024-11-17 05:32:30--  https://dumps.wikimedia.org/kowikisource/20241101/kowikisource-20241101-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151135574 (144M) [application/octet-stream]
Saving to: ‘kowikisource-20241101-pages-articles.xml.bz2’

kowikisource-202411 100%[===================>] 144.13M  4.54MB/s    in 31s     

2024-11-17 05:33:02 (4.58 MB/s) - ‘kowikisource-20241101-pages-articles.xml.bz2’ saved [151135574/151135574]



In [4]:
# Parsing (3분 가량 걸림)
!python -m wikiextractor.WikiExtractor kowikisource-20241101-pages-articles.xml.bz2

INFO: Preprocessing 'kowikisource-20241101-pages-articles.xml.bz2' to collect template definitions: this may take some time.
INFO: Loaded 1242 templates in 69.9s
INFO: Starting page extraction from kowikisource-20241101-pages-articles.xml.bz2.
INFO: Using 1 extract processes.
INFO: Finished 1-process extraction of 37057 articles in 154.3s (240.2 art/s)


In [5]:
# 파싱한 파일을 하나의 xml로 합치기
!cat text/*/* > kowikisource.xml

In [6]:
# 맨위에 추가
!echo "$(echo -n -e '<?xml version="1.0" encoding="UTF-8"?>\n<wikisource>\n'; cat kowikisource.xml)" > kowikisource.xml

In [7]:
!head -n 20 kowikisource.xml # 첫 20줄만 확인

<?xml version="1.0" encoding="UTF-8"?>
<wikisource>
<doc id="1" url="https://ko.wikisource.org/wiki?curid=1" title="애국가 (대한민국)">
애국가 (대한민국)

한자 혼용
한글

</doc>
<doc id="2" url="https://ko.wikisource.org/wiki?curid=2" title="대한민국헌법 (한자혼용)">
대한민국헌법 (한자혼용)



</doc>
<doc id="3" url="https://ko.wikisource.org/wiki?curid=3" title="조선민주주의인민공화국 사회주의헌법">
조선민주주의인민공화국 사회주의헌법

서문.
조선민주주의인민공화국은 위대한 김일성동지와 김정일동지의 사상과 령도를 구현한 주체의 사회주의조국이다.


In [8]:
# 맨 아랫줄에 추가
!echo '</wikisource>' >> kowikisource.xml

In [9]:
!tail kowikisource.xml # 마지막줄 확인



</doc>
<doc id="93454" url="https://ko.wikisource.org/wiki?curid=93454" title="에티카 (스피노자)">
에티카 (스피노자)



</doc>
</wikisource>


##### 위키문헌 다루기

In [10]:
# xml 파일 dataframe로 읽기
df = pd.read_xml('./kowikisource.xml')
df

,id,url,title,doc
0,1,https://ko.wikisource.org/wiki?curid=1,애국가 (대한민국),\n애국가 (대한민국)\n\n한자 혼용\n한글\n\n
1,2,https://ko.wikisource.org/wiki?curid=2,대한민국헌법 (한자혼용),\n대한민국헌법 (한자혼용)\n\n\n\n
2,3,https://ko.wikisource.org/wiki?curid=3,조선민주주의인민공화국 사회주의헌법,\n조선민주주의인민공화국 사회주의헌법\n\n서문.\n조선민주주의인민공화국은 위대한 ...
3,7,https://ko.wikisource.org/wiki?curid=7,한글 맞춤법 (제88-1호),\n한글 맞춤법 (제88-1호)\n\n제1장 총 칙.\n제 1 항 한글 맞춤법은 표...
4,14,https://ko.wikisource.org/wiki?curid=14,"민법 (대한민국, 제8720호)","\n민법 (대한민국, 제8720호)\n\n\n\n"
...,...,...,...,...
37051,93429,https://ko.wikisource.org/wiki?curid=93429,선조국문교서,\n선조국문교서\n\n\n\n
37052,93439,https://ko.wikisource.org/wiki?curid=93439,아리사 버들,\n아리사 버들\n\n\n\n
37053,93452,https://ko.wikisource.org/wiki?curid=93452,엥케이리디온,\n엥케이리디온\n\n\n\n
37054,93453,https://ko.wikisource.org/wiki?curid=93453,에티카 (스피노자)/신에 대하여,\n에티카 (스피노자)/신에 대하여\n\n\n\n


In [11]:
def process_getPoemLines(data):
    """
    주어진 리스트에서 불필요한 인자를 제거하고, 제목과 본문으로 분리하여 DataFrame 생성

    Parameters:
    - data (list): 제목과 본문 데이터가 포함된 리스트

    Returns:
    - DataFrame: '본문' 열이 먼저, 'title' 열이 뒤에 오는 형태의 데이터프레임
    """
    # 불필요한 인자 제거
    filtered_list = [item for item in data if item not in [None, "", " ", [], {}, '<poem>', '</poem>', '진달래꽃 (시집)/']]

    # 제목 편집 (0번 인자 수정)
    pick_title = filtered_list[0].split('/')
    pick_title = pick_title[1] if len(pick_title) > 1 else filtered_list[0]
    filtered_list[0] = pick_title

    # 'title'은 첫 번째 인자, '본문'은 나머지 인자들
    title = filtered_list[0]
    body = filtered_list[1:]

    # DataFrame 생성: 본문 열이 먼저 오도록
    df = pd.DataFrame({'본문': body, 'title': [title] * len(body)})

    return df

In [31]:
def update_results_with_title(df, results_df, process_function, title_to_search, author):

    # 제목에 해당하는 텍스트 검색
    title_to_search.replace("_", " ")
    filtered_text = df[df['title'].str.contains(title_to_search, na=False)]
    if filtered_text.empty:
        raise ValueError(f"'{title_to_search}'에 해당하는 데이터가 없습니다.")

    # 'doc' 값 추출
    # text_value = filtered_text.iloc[0]['doc']

    # 텍스트 처리
    # processed_result = process_function(text_value.split('\n'))

    # 처리 결과를 기존 DataFrame에 추가
    # updated_results = pd.concat([results_df, pd.DataFrame(processed_result)], ignore_index=True)

    # 조건 반복문으로 'doc' 처리
    for index, row in filtered_text.iterrows():
        if 'doc' in row:  # 'doc' 열이 존재하는지 확인
            # 'doc' 값 추출
            text_value = row['doc']

            # 텍스트 처리
            processed_result = process_function(text_value.split('\n'))

            # 처리 결과를 기존 DataFrame에 추가
            results_df = pd.concat([results_df, pd.DataFrame(processed_result)], ignore_index=True)

            # # 새로운 열 추가 (고정된 값)
            results_df['저자'] = author
        else:
            print(f"Row {index}에 'doc' 값이 없습니다. 건너뜁니다.")
    return results_df

In [30]:
# 검색 데이터 확인.
check = df[df['title'].str.contains('풀따기')]
check

,id,url,title,doc
14347,34213,https://ko.wikisource.org/wiki?curid=34213,풀따기,\n풀따기\n\n\n\n
14383,34252,https://ko.wikisource.org/wiki?curid=34252,진달래꽃/풀따기,\n진달래꽃/풀따기\n\n\n\n


In [16]:
check.to_excel('check.xlsx')

In [34]:
text1 = pd.DataFrame(columns=['본문', '제목'])

In [32]:
text1 = update_results_with_title(df, text1, process_getPoemLines, '진달래꽃', '김소월')
text1.iloc[-1] # 마지막줄 확인

,4886
본문,어이없이도 나의 영(靈)은 혼자 울고 있어라.
title,새빨간 핏빛의 진달래꽃이 질 때
저자,김소월


In [33]:
# 엑셀로 저장하기
text1.to_excel('위키문헌_김소월.xlsx')

In [20]:
def update_results_with_title(df, results_df, title_to_search, process_function, author):

    # 제목에 해당하는 텍스트 검색
    title_to_search.replace("_", " ")
    filtered_text = df[df['title'].str.contains(title_to_search, na=False)]
    if filtered_text.empty:
        raise ValueError(f"'{title_to_search}'에 해당하는 데이터가 없습니다.")

    # 'doc' 값 추출
    text_value = filtered_text.iloc[0]['doc']

    # 텍스트 처리
    processed_result = process_function(text_value.split('\n'))

    # 처리 결과를 기존 DataFrame에 추가
    updated_results = pd.concat([results_df, pd.DataFrame(processed_result)], ignore_index=True)

    # 새로운 열 추가 (고정된 값)
    updated_results['저자'] = author

    return updated_results